In [29]:
import Table_Reader as TR
import LCCall as LCC
import LCAnalyses as LCA
import LCPlot as LCP
import importlib
import matplotlib.pyplot as plt
import scipy.special as fc
import random as rd
import numpy as np
importlib.reload(LCC)
importlib.reload(LCA)
importlib.reload(LCP)
importlib.reload(TR)

<module 'Table_Reader' from '/media/lyhee/Secdrive/TSmain/Pip_test/trigger_codes_github/Table_Reader.py'>

In [10]:
print(fc.gammainc(0.5,0.5*1**2))
print(fc.gammaincinv(0.5,0.8))

0.6826894921370859
0.821187207574909


In [11]:
fmean = 1.4
np.sqrt(0.014**2 + (fmean*0.02)**2)/fmean

0.0223606797749979

In [30]:
input_dir = "/mnt/Secdrive/TSmain/TSmain/"
rgs = ['IC348','NGC1333','NGC2024','NGC2068','OMC23','OPHCORE','SERPM','SERPS']
rra  =[56.075, 52.225, 85.42083, 86.55417, 83.87917, 246.77083, 277.45417, 277.50833]
rdec =[32.08306, 31.28111, -1.8975, -0.10139, -5.01056, -24.54361, 1.25556, -2.04667]
#indices = np.where(np.sqrt(np.array(data.RAs)**2+np.array(data.DECs)**2)/60 < 18)
%matplotlib tk
output_dir= input_dir+"Completeness_lin/"

# 5-10 sec for SERPM for 1000 iter
# ~0.6 min for OPHCORE for 10000 iter

In [21]:
writing=1
test = 0
rgn = 6
query = []
Rst=[]

#L = [k,freq,phase/2/np.pi,H_mean[-1],T1,T2,T3*2,A_n[0],A_n[1],A_n[2],chi_Null]
for rgn in range(0,8):
    #for freq in [0.1, 0.15, 0.2, 0.5,1,5]:

    if writing:
        file = open(output_dir+rgs[rgn]+".tab",'w')
        file.write("#Index"+'\t'+ "C"+'\t'+
               "T1"+'\t'+"T2"+'\t'+"T3"+'\t'+"A2"+'\t'+"A3"+'\t'+'\t'+"A4"+'\t'+
               "chi2_null"+'\t'+ "chi2_null_det"+"\n")
    Index = 1
    counter = 1
    plot = 0
    Iter = 10000
    thr = np.array([1e-2,1e-3,1e-4])
    #output_dir = input_dir+"IfyouNeed/"
    for i in range(rgn,rgn+1):
        print(rgs[i])
        data = TR.ReadTable(input_dir+'Data/'+rgs[i]+'_source_data.dat')
        indices = [int(x) for x in data.Indices]
        data.JDs = (np.array(data.JDs) - data.JDs[0])/365.25 # JD to relative yr
        for k in range(0,Iter):
            #freq = rd.uniform(freq_in[0],freq_in[1])
            #freq = 0.15

            #r_period = (np.max(data.JDs)-np.min(data.JDs))/freq
            #freq = (np.max(data.JDs)-np.min(data.JDs))/period
            #A_fid = np.exp(rd.uniform(A_fid_in[0],A_fid_in[1]))
            #phase = rd.uniform(phase_in[0],phase_in[1])

            j = 0

            #phases = np.linspace(0,2*np.pi,10)[:-1]

            #modelpar = [mean[jy/beam], freq[day], amp[jy/beam], iphase]

            #fid_err = np.sqrt(0.014**2+(0.02*data.mean_pfluxes[j])**2)

            num = len(data.pfluxes[j])
            H_data = []
            G = []
            H_real = []
            H_mean = []
            for x in data.JDs:
                G.append(rd.gauss(0,1))
                H_real.append(x)
                H_data.append(H_real[-1]+G[-1])
            data.pfluxes[j] = np.array(H_data)
            H_real = np.array(H_real)
            H_mean.append(np.mean(H_real))
            H_real = H_real-np.mean(H_real)-np.mean(G)
            G = np.array(G)
            T3 = np.sum(G**2)/num
            T1 = np.sum((H_real)**2)/num
            T2 = np.sum(G*H_real)/num
            #T4 = A_fid


            if not test:
                #Null Hypothesis
                chi2_Null = np.sum(abs(data.pfluxes[j]-np.mean(data.pfluxes[j]))**2)

                #Periodogram
                #ls_source = LCA.LCLombscargle(data.JDs,data.pfluxes[j],1)
                #modelpar = ls_source.modelpara()
                #FAP_Sin = ls_source.FAPs[np.where(ls_source.powers == np.max(ls_source.powers))][0]
                JD_center = (min(data.JDs) + max(data.JDs)) / 2
                #Power_Sin = np.max(ls_source.powers)
                #P_Null = fc.gammainc((num-1)/2, chi2_Null/2)

                #num_freqs=int(freq/dfreq)+1
                alphas= (thr)**(-2/(num-2))
                A_n = []
                for alpha in alphas:
                    a = T1
                    b = T2*2
                    c = T3*(1-alpha)
                    A_n.append((-b+np.sqrt(b**2-4*a*c))/(2*a))

                #Null Hypothesis when detected
                chi2_Null_det = a*A_n[-1]**2+b*A_n[-1]+c
                #if any(sum([checker1,checker2,checker3])): writing =1


            L = [k, H_mean[-1], T1
                 , T2*2, T3, A_n[0], A_n[1], 
                 A_n[2], chi2_Null/num,chi2_Null_det/num]

            Rst.append(L)
            L = [str(x)+'\t' for x in L[:-1]]+[str(L[-1])]
            if writing:
                file.writelines(L)
                file.write('\n')
            Index += 1
        if writing:
            file.close()
        #print(freq)
        print("DONE!")
#query = set(query)

IC348
DONE!


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/Secdrive/TSmain/TSmain/IfyouNeed/NGC1333.tab'

In [298]:
H_data = []
G = []
H_real = []
H_mean = []
for x in data.JDs:
    G.append(rd.gauss(0,1))
    H_real.append(5*np.sin(2*np.pi*(0.1*x+0.0)))
    H_data.append(H_real[-1]+G[-1])
data.pfluxes[j] = np.array(H_data)

plt.figure()
plt.scatter(data.JDs,data.pfluxes[j])
plt.scatter(data.JDs,H_real)

In [36]:
#output_dir = input_dir+'Completeness/'
rgn=5
a0 = np.loadtxt(output_dir + rgs[0] +".tab")
a1 = np.loadtxt(output_dir + rgs[1] +".tab")
a2 = np.loadtxt(output_dir + rgs[2] +".tab")
a3 = np.loadtxt(output_dir + rgs[3] +".tab")
a4 = np.loadtxt(output_dir + rgs[4] +".tab")
a5 = np.loadtxt(output_dir + rgs[5] +".tab")
a6 = np.loadtxt(output_dir + rgs[6] +".tab")
a7 = np.loadtxt(output_dir + rgs[7] +".tab")
a0 = TR.Columnread(a0)
a1 = TR.Columnread(a1)
a2 = TR.Columnread(a2)
a3 = TR.Columnread(a3)
a4 = TR.Columnread(a4)
a5 = TR.Columnread(a5)
a6 = TR.Columnread(a6)
a7 = TR.Columnread(a7)





In [97]:

a0_thr,a0_thr68, a0_thr50, a0_thr01 = np.sort(a0[7])[9900], np.sort(a0[7])[6800], np.sort(a0[7])[5000], np.sort(a0[7])[100]
a1_thr,a1_thr68, a1_thr50, a1_thr01 = np.sort(a1[7])[9900], np.sort(a1[7])[6800], np.sort(a1[7])[5000], np.sort(a1[7])[100]
a2_thr,a2_thr68, a2_thr50, a2_thr01 = np.sort(a2[7])[9900], np.sort(a2[7])[6800], np.sort(a2[7])[5000], np.sort(a2[7])[100]
a3_thr,a3_thr68, a3_thr50, a3_thr01 = np.sort(a3[7])[9900], np.sort(a3[7])[6800], np.sort(a3[7])[5000], np.sort(a3[7])[100]
a4_thr,a4_thr68, a4_thr50, a4_thr01 = np.sort(a4[7])[9900], np.sort(a4[7])[6800], np.sort(a4[7])[5000], np.sort(a4[7])[100]
a5_thr,a5_thr68, a5_thr50, a5_thr01 = np.sort(a5[7])[9900], np.sort(a5[7])[6800], np.sort(a5[7])[5000], np.sort(a5[7])[100]
a6_thr,a6_thr68, a6_thr50, a6_thr01 = np.sort(a6[7])[9900], np.sort(a6[7])[6800], np.sort(a6[7])[5000], np.sort(a6[7])[100]
a7_thr,a7_thr68, a7_thr50, a7_thr01 = np.sort(a7[7])[9900], np.sort(a7[7])[6800], np.sort(a7[7])[5000], np.sort(a7[7])[100]


In [99]:

print([a0_thr,a0_thr68, a0_thr50, a0_thr01])
print([a1_thr,a1_thr68, a1_thr50, a1_thr01])
print([a2_thr,a2_thr68, a2_thr50, a2_thr01])
print([a3_thr,a3_thr68, a3_thr50, a3_thr01])
print([a4_thr,a4_thr68, a4_thr50, a4_thr01])
print([a5_thr,a5_thr68, a5_thr50, a5_thr01])
print([a6_thr,a6_thr68, a6_thr50, a6_thr01])
print([a7_thr,a7_thr68, a7_thr50, a7_thr01])
print(np.mean([a0_thr,a1_thr,a2_thr,a3_thr,a4_thr,a5_thr,a6_thr,a7_thr]))
print(np.mean([a0_thr68,a1_thr68,a2_thr68,a3_thr68,a4_thr68,a5_thr68,a6_thr68,a7_thr68]))

print(np.mean([a0_thr01,a1_thr01,a2_thr01,a3_thr01,a4_thr01,a5_thr01,a6_thr01,a7_thr01]))

[1.1516945856226846, 0.7605933134332495, 0.6817261102696921, 0.3987472124760213]
[1.134486360871549, 0.7541300025799637, 0.6752757331885132, 0.3945198651935258]
[1.1044116526554133, 0.7513360039377689, 0.6702717747390778, 0.39313254583238255]
[1.0930242365648666, 0.7379446433629253, 0.6617961984640698, 0.39218858856117245]
[1.1776904665203012, 0.7886716792938105, 0.7076720096797632, 0.4197295868504352]
[1.3884821116989043, 0.9354584778197295, 0.8397854826681016, 0.48899243211847016]
[1.0514092740462166, 0.7037091526745984, 0.6306132732313603, 0.36886172641349174]
[1.147937960356929, 0.7662426053312442, 0.6894059352623677, 0.4075349905931033]
1.1561420810421081
0.7747607348041613
0.4079633685048253


In [83]:
plt.close()

a = TR.Columnread(Rst)
fig, ax = plt.subplots(2,1)
for axis in ax:
    axis.hist(a0[7],alpha=1)
    #axis.hist(a1[7],alpha=0.3)
    #axis.hist(a2[7],alpha=0.3)
    #axis.hist(a3[7],alpha=0.3)
    #axis.hist(a4[7],alpha=0.3)
    #axis.hist(a5[7],alpha=0.3)
    #axis.hist(a6[7],alpha=0.3)
    #axis.hist(a7[7],alpha=0.3)
    axis.set_ylim([0,4000])
#ax[0].plot([a7_thr68,a7_thr68],[0,4000],color='red')
#ax[0].plot([a6_thr68,a6_thr68],[0,4000],color='orange')
#ax[0].plot([a5_thr68,a5_thr68],[0,4000],color='yellow')
#ax[0].plot([a4_thr68,a4_thr68],[0,4000],color='green')
#ax[0].plot([a3_thr68,a3_thr68],[0,4000],color='blue')
#ax[0].plot([a2_thr68,a2_thr68],[0,4000],color='navy')
#ax[0].plot([a1_thr68,a1_thr68],[0,4000],color='purple')
ax[0].plot([a0_thr68,a0_thr68],[0,4000],color='black')
#ax[1].plot([a7_thr,a7_thr],[0,4000],color='red')
#ax[1].plot([a6_thr,a6_thr],[0,4000],color='orange')
#ax[1].plot([a5_thr,a5_thr],[0,4000],color='yellow')
#ax[1].plot([a4_thr,a4_thr],[0,4000],color='green')
#ax[1].plot([a3_thr,a3_thr],[0,4000],color='blue')
#ax[1].plot([a2_thr,a2_thr],[0,4000],color='navy')
#ax[1].plot([a1_thr,a1_thr],[0,4000],color='purple')
ax[1].plot([a0_thr,a0_thr],[0,4000],color='black')
ax[1].set_xlabel(r"$\sigma_{fid}$")
ax[1].annotate("99%: 1.15",xy=[0.7,0.3], xycoords='figure fraction')
ax[0].annotate("68%: 0.76",xy=[0.7,0.8], xycoords='figure fraction')
#ax.scatter(a1[0][2],a1[6][2],alpha=0.1)
#ax.scatter(a1[0][2],a1[5][2],alpha=0.1)

Text(0.7, 0.8, '68%: 0.76')

In [95]:
plt.close()

a = TR.Columnread(Rst)
fig, ax = plt.subplots()
ax.hist(a0[7],alpha=1)
ax.set_ylim([0,4000])
ax.plot([a0_thr50,a0_thr50],[0,4000],color='grey')
ax.plot([a0_thr68,a0_thr68],[0,4000],color='blue')
ax.plot([a0_thr,a0_thr],[0,4000],color='red')
ax.set_xlabel(r"$\sigma_{fid}$")
ax.annotate("50%: 0.68",xy=[0.3,0.8], xycoords='figure fraction')
ax.annotate("68%: 0.76",xy=[0.48,0.8], xycoords='figure fraction')
ax.annotate("99%: 1.15",xy=[0.7,0.8], xycoords='figure fraction')

Text(0.48, 0.8, '68%: 0.76')

In [72]:
output_dir

'/mnt/Secdrive/TSmain/TSmain/Completeness_lin/'

In [81]:
plt.savefig(output_dir+"Linear_Completeness_68_99"+".pdf")

In [96]:
plt.savefig(output_dir+"Linear_Completeness_IC348"+".pdf")

In [8]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Phase')
ax.set_ylabel('Sigma Level')
ax.scatter(a1[2],a1[8],alpha=0.3,edgecolors='none',label=r'P=12')
ax.scatter(a3[2],a3[8],alpha=0.3,edgecolors='none',label=r'P=8')
ax.scatter(a5[2],a5[8],alpha=0.3,edgecolors='none',label=r'P=2')
ax.legend()

In [152]:
a6_thr = np.sort(a6[8])[9900]
print(a6_thr)

2.14381044354215


In [282]:
fig, ax = plt.subplots(2,2)

x = a7[2]
y = a7[8]
x2 = a8[2]
y2 = a8[8]
x3 = a1[2]
y3 = a1[8]
x4 = a4[2]
y4 = a4[8]

ax[0][0].scatter(x, y,marker='o',color='grey',edgecolors='none',alpha=0.2,s=20
           ,label=r'P = 40 yr')
ax[0][1].scatter(x2, y2,marker='o',color='orange',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 24 yr')
ax[1][0].scatter(x3, y3,marker='o',color='green',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 12 yr')
ax[1][1].scatter(x4, y4,marker='o',color='lightsteelblue',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 4 yr')

for axis in ax:
    axis[0].label_outer()
    axis[1].label_outer()
    #axis[0].annotate(r"Sub-mm",xy=[0.35,0.8],xycoords='figure fraction',fontsize=15
    #axis[0].set_ylabel(r"Detectable Amp. [$\sigma$]")
    #axis[1].set_ylabel(r"Detectable Amp. [$\sigma$]")
    axis[0].legend()
    axis[1].legend()
plt.tight_layout()

In [256]:
axis

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f656e772090>,
      dtype=object)

In [2]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
# the random data

x = a1[2]
y = a1[8]
x2 = a2[2]
y2 = a2[8]
x3 = a3[2]
y3 = a3[8]
x4 = a5[2]
y4 = a5[8]


x = a7[2]
y = a7[8]
x2 = a8[2]
y2 = a8[8]
x3 = a1[2]
y3 = a1[8]
x4 = a4[2]
y4 = a4[8]

thr1 = np.sort(y)[9900]
thr1_2 = np.sort(y)[6800]
thr2 = np.sort(y2)[9900]
thr3 = np.sort(y3)[9900]
thr4 = np.sort(y4)[9900]
print(a1_thr,a2_thr,a3_thr)
fig, ax = plt.subplots(figsize=(8, 8))

# the scatter plot:
ax.scatter(x, y,marker='o',color='grey',edgecolors='none',alpha=0.2,s=20
           ,label=r'P = 40 yr')
ax.scatter(x2, y2,marker='o',color='orange',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 24 yr')
ax.scatter(x3, y3,marker='o',color='green',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 12 yr')
ax.scatter(x4, y4,marker='o',color='lightsteelblue',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 4 yr')
ax.plot([0,1],[thr1,thr1],'2--',color='grey')
ax.plot([0,1],[thr2,thr2],'2--',color='orange')
ax.plot([0,1],[thr3,thr3],'2--',color='green')
ax.plot([0,1],[thr4,thr4],'2--',color='blue')
#ax.scatter()

ax.set_xlim(-0.02,1.02)
ax.set_ylim(0,20)
ax.set_xlabel(r"Phase",fontsize=12)
ax.set_ylabel(r"Detectable Amplitude ($\sigma$)",fontsize=12)
ax.legend(loc='upper right')
# Set aspect of the main axes.
#ax.set_aspect(0.2)

# create new axes on the right and on the top of the current axes
divider = make_axes_locatable(ax)
# below height and pad are in inches
#ax_histx = divider.append_axes("top", 1.0, pad=0.1, sharex=ax)
ax_histy = divider.append_axes("right", 1.0, pad=0.2, sharey=ax)

# make some labels invisible
#ax_histx.xaxis.set_tick_params(labelbottom=False)
#ax_histx.xaxis.set_xscale("log")
ax_histy.yaxis.set_tick_params(labelleft=False)
#ax_histy.yaxis.set_yscale("log")

# now determine nice limits by hand:
#binwidth = 0.05
binwidth = 0.3
xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
lim = (int(xymax/binwidth) + 1)*binwidth

bins = np.arange(-lim, lim + binwidth, binwidth)
#ax_histx.hist(x, bins=bins,histtype='step',color='black')
ax_histy.hist(y, bins=bins,orientation='horizontal',histtype='step',color='black'
             ,linewidth=2,density=0)
ax_histy.plot([0,5000],[thr1,thr1],color='black')
ax_histy.hist(y2, bins=bins,orientation='horizontal',histtype='step',color='orange'
             ,linewidth=2,density=0)
ax_histy.plot([0,5000],[thr2,thr2],color='orange')
ax_histy.hist(y3, bins=bins,orientation='horizontal',histtype='step',color='green'
             ,linewidth=2,density=0)
ax_histy.plot([0,5000],[thr3,thr3],color='green')
ax_histy.hist(y4, bins=bins,orientation='horizontal',histtype='step',color='blue'
             ,linewidth=2,density=0)
ax_histy.plot([0,5000],[thr4,thr4],color='blue')
#ax_histy.plot([0,5000],[thr2,a4_thr],color='blue')
#ax_histy.plot([0,5000],[thr3,a5_thr],color='violet')

# the xaxis of ax_histx and yaxis of ax_histy are shared with ax,
# thus there is no need to manually adjust the xlim and ylim of these
# axis.
#ax_histx.set_xscale("log")
#ax_histy.set_yscale("log")

#ax_histx.set_yscale("log")
#ax_histx.set_ylim([0.8, 110])
#ax_histy.set_xscale("log")
ax_histy.set_xlim([0, 5000])
#ax.set_ylim(5e-3,15)
#ax.set_xlim(0.13,15)
#ax_histx.set_ylim(0.13,15)
#ax.plot([0.13,15],[5e-3,15],ls='--',color='grey')
#ax_histy.set_xlim(5e-3,15)
#ax_histx.set_yticks([0, 30, 60])
#ax_histy.set_xticks([0, 30, 60])
plt.show()

NameError: name 'a1' is not defined

In [297]:
plt.savefig(output_dir+"Phase-DAmp-99per.pdf")

In [191]:
rgn=5
a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
a3 = np.loadtxt(output_dir + rgs[rgn] +"_period6.tab")
a4 = np.loadtxt(output_dir + rgs[rgn] +"_period4.tab")
a5 = np.loadtxt(output_dir + rgs[rgn] +"_period2.tab")
a6 = np.loadtxt(output_dir + rgs[rgn] +"_period1.tab")
a1 = TR.Columnread(a1)
a2 = TR.Columnread(a2)
a3 = TR.Columnread(a3)
a4 = TR.Columnread(a4)
a5 = TR.Columnread(a5)
a6 = TR.Columnread(a6)
a1_thr = np.sort(a1[8])[9900]
a2_thr = np.sort(a2[8])[9900]
a3_thr = np.sort(a3[8])[9900]
a4_thr = np.sort(a4[8])[9900]
a5_thr = np.sort(a5[8])[9900]
a6_thr = np.sort(a6[8])[9900]
#print(a1_thr,a2_thr,a3_thr,a4_thr,a5_thr)

4.304044731660869 2.6727313323731168 2.1264513357245467 2.356224133979633 2.503408287345745


#a1 freq0.1_test.tab")
#a6 freq0.15_test.tab")
#a5 freq0.2.tab")
#a4 freq0.5.tab")
#a2 freq1.tab")
#a3 freq5.tab")

In [14]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Period [yr]')
ax.set_ylabel(r'Confident Detection level [$\sigma$]')
ax.set_ylim([1.2,6])
xfreqs =np.array([0.1,0.15,0.2,0.5,1,5])
xperiods = np.array([12,8,6,4,2,1])

for rgn in range(0,8):
    a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
    a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
    a3 = np.loadtxt(output_dir + rgs[rgn] +"_period6.tab")
    a4 = np.loadtxt(output_dir + rgs[rgn] +"_period4.tab")
    a5 = np.loadtxt(output_dir + rgs[rgn] +"_period2.tab")
    a6 = np.loadtxt(output_dir + rgs[rgn] +"_period1.tab")
    a7 = np.loadtxt(output_dir + rgs[rgn] +"_period40.tab")
    a8 = np.loadtxt(output_dir + rgs[rgn] +"_period24.tab")
    
    a1 = TR.Columnread(a1)
    a2 = TR.Columnread(a2)
    a3 = TR.Columnread(a3)
    a4 = TR.Columnread(a4)
    a5 = TR.Columnread(a5)
    a6 = TR.Columnread(a6)
    a7 = TR.Columnread(a7)
    a8 = TR.Columnread(a8)
    a1_thr = np.sort(a1[8])[9900]
    a2_thr = np.sort(a2[8])[9900]
    a3_thr = np.sort(a3[8])[9900]
    a4_thr = np.sort(a4[8])[9900]
    a5_thr = np.sort(a5[8])[9900]
    a6_thr = np.sort(a6[8])[9900]
    a7_thr = np.sort(a7[8])[9900]
    a8_thr = np.sort(a8[8])[9900]
    
    a1_thr2 = np.sort(a1[8])[6800]
    a2_thr2 = np.sort(a2[8])[6800]
    a3_thr2 = np.sort(a3[8])[6800]
    a4_thr2 = np.sort(a4[8])[6800]
    a5_thr2 = np.sort(a5[8])[6800]
    a6_thr2 = np.sort(a6[8])[6800]
    a7_thr2 = np.sort(a7[8])[6800]
    a8_thr2 = np.sort(a8[8])[6800]
    
    print(rgs[rgn])
    print("99%")
    print([a7_thr,a1_thr,a2_thr,a3_thr,a4_thr,a5_thr,a6_thr])
    print("68%")
    print([a7_thr2,a1_thr2,a2_thr2,a3_thr2,a4_thr2,a5_thr2,a6_thr2])
    #print(np.where(np.sort(a1[8])<a4_thr)[-1])
    #print(np.where(np.sort(a2[8])<a4_thr)[-1])
    thrs2= np.array([a1_thr2,a2_thr2,a3_thr2,a4_thr2,a5_thr2,a6_thr2])
    thrs = np.array([a1_thr,a2_thr,a3_thr,a4_thr,a5_thr,a6_thr])
    ax.errorbar(xperiods,thrs,fmt='o-',label=rgs[rgn])
ax.legend()

IC348
99%
[18.12667122430125, 4.363723865021905, 2.6564658574561455, 1.963110908145642, 1.8633787432096465, 1.9570742328548625, 2.113938967958012]
68%
[4.214606787559613, 2.0099895532961747, 1.4396597823282016, 1.1920637264717708, 1.1625454074232229, 1.2360382122685971, 1.3267304197998113]
NGC1333
99%
[17.930728478234045, 4.485424436740078, 2.725347312496146, 2.0001896496889295, 1.7738357933322366, 1.902602758487375, 2.050209410310074]
68%
[4.23464952099503, 2.013866900204387, 1.4238258956704817, 1.190035093070205, 1.1332324772791138, 1.2033654145553994, 1.305658911442305]
NGC2024
99%
[19.17325291462955, 4.482956849815358, 2.796136720447572, 2.0224365319698303, 1.8067875089153251, 2.03822168088007, 2.1282718442157886]
68%
[4.16205014231341, 1.987152444271198, 1.4242518141986096, 1.1799625804112832, 1.150433778814641, 1.2394881901402761, 1.358912019487329]
NGC2068
99%
[18.38806850456431, 4.391157401085747, 2.6129704911670157, 1.9960797500195304, 1.8002743349549006, 1.9465375367591085, 2

In [10]:
plt.savefig(input_dir+"Completeness/"+"map_CDl.pdf")

In [ ]:
for rgn in range(0,8):
    a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
    a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
    a1 = TR.Columnread(a1)
    a2 = TR.Columnread(a2)

In [139]:
plt.savefig(input_dir+"Completeness/"+"Phase-Amp_diagram_more.pdf")

In [108]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Chi-null')
ax.set_ylabel('Number')
binwidth = 0.1
xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
lim = (int(xymax/binwidth) + 1)*binwidth

bins = np.arange(-lim, lim + binwidth, binwidth)
ax.hist(a1[11], histtype='step',color='black'
             ,linewidth=2,label=r'P=10$\tau$')
ax.hist(a2[11], histtype='step',color='orange'
             ,linewidth=2,label=r'P=1$\tau$')
ax.hist(a3[11],histtype='step',color='green'
             ,linewidth=2,label=r'P=0.2$\tau$')
ax.legend()

In [111]:
plt.savefig(input_dir+"Completeness/"+"chi2_null_hist.pdf")

In [217]:
plt.close()
fig, ax = plt.subplots()
x = np.linspace(0.14,1.4,101)

ax.plot(x, np.sqrt(0.014**2+(0.02*x)**2))
ax.plot(x, 2*np.sqrt(0.014**2+(0.02*x)**2))
ax.plot(x, 3*np.sqrt(0.014**2+(0.02*x)**2))

ax.set_xscale('log')

In [110]:
a1[11][-1]

1.1022966728098704